<h1><b>Restricted Boltzmann Machine και Deep Belief Network</b></h1>
<p align="justify">Στην συγκεκριμένη άσκηση θα μελετήσετε τον τρόπο λειτουργίας μιας <i>RBM (<a href="https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine">Restricted Boltzmann Machine</a>)</i> καθώς και των <i>DBN (<a href="https://en.wikipedia.org/wiki/Deep_belief_network">Deep Belief Network</a>)</i>, χρησιμοποιώντας το έτοιμο πρόγραμμα που σας δίνεται.Το συγκεκριμένο πρόγραμμα αξιοποιεί το <a href="https://en.wikipedia.org/wiki/MNIST_database">dataset του <i>MNIST</i></a>, όπου είναι μια μεγάλη βάση δεδομένων με χειρόγραφα ψηφία που χρησιμοποιείται συνήθως για την εκπαίδευση διαφόρων συστημάτων επεξεργασίας εικόνας. Για την άσκηση, θα πρέπει να χρησιμοποιήσετε το αρχείο <i>mnist_original.mat</i>, το οποίο είναι διαθέσιμο από <a href="https://www.kaggle.com/datasets/avnishnish/mnist-original?resource=download">εδώ</a>.</p>
<p align="justify">Μία αρκετά σημαντική εφαρμογή της <i>RBM</i> είναι η εξαγωγή χαρακτηριστικών (feature representation) από ένα dataset με σκοπό την αναπαράσταση της εισόδου (ορατοί νευρώνες) με ένα διάνυσμα μικρότερης διάστασης (κρυφοί νευρώνες). Στη συγκεκριμένη άσκηση θα συγκρίνετε την ακρίβεια ενός ταξινομητή ψηφίων με τη χρήση του αλγορίθμου <i>Logistic Regression</i>, όταν εκείνος δέχεται ως είσοδο το dataset (i) χωρίς να έχει υποστεί επεξεργασία από το <i>RBM</i>, (ii) αφου υποστεί επεξεργασία από το <i>RBM</i>, (iii) με τη χρήση <i>DBN</i>, δηλαδή δύο stacked <i>RBM</i>.</p>
<p align="justify"> Με βάση τον κώδικα που σας έχει δοθεί, καλείστε να απαντήσετε στα παρακάτω ερωτήματα:</p>
<ul>
<li>Να περιγράψετε σύντομα τον τρόπο λειτουργίας μιας <i>RBM</i>. Τι διαφορές έχει σε σχέση με μία <i> Μηχανή Boltzmann</i>;</li>
<li>Ποια είναι η λογική των <i>DBN</i> και σε τι προβλήματα τα αξιοποιούμε;</li>
<li>Να αναφέρετε τις βασικότερες εφαρμογές των <i>RBM</i> και <i>DBN</i>.</li>
<li>Εκτός από <i>RBM</i>, τι άλλα μοντέλα μπορούν να χρησιμοποιηθούν για να δημιουργήσουν <i>DBN</i>.</li>
<li>Συγκρίνετε τα αποτελέσματα της ταξινόμησης με τον αλγόριθμo <i>Logistic Regression</i> χωρίς τη χρήση <i>RBM</i> σε σχέση με τα αποτελέσματα της ταξινόμησης που έχει χρησιμοποιηθεί η <i>RBM</i> καθώς και με αυτή όπου χρησιμοποιούνται <i>RBM</i> και <i>DBN</i> για την εξαγωγή των χαρακτηριστικών. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;</li>
</ul>

In [8]:
#!/usr/bin/env python
# source: https://devdreamz.com/question/905929-stacking-rbms-to-create-deep-belief-network-in-sklearn

import numpy as np
import matplotlib.pyplot as plt

from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import BernoulliRBM
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

def norm(arr):
    arr = arr.astype(float)
    arr -= arr.min()
    arr /= arr.max()
    return arr

if __name__ == '__main__':

    # load MNIST data set
    mnist = loadmat("mnist-original.mat")
    X, Y = mnist["data"].T, mnist["label"][0]

    # normalize inputs to 0-1 range
    X = norm(X)

    # split into train, validation, and test data sets
    X_train, X_test, Y_train, Y_test = train_test_split(X,       Y,       test_size=10000, random_state=0)
    X_train, X_val,  Y_train, Y_val  = train_test_split(X_train, Y_train, test_size=10000, random_state=0)

    # --------------------------------------------------------------------------------
    # set hyperparameters

    learning_rate = 0.02 
    total_units   =  800 
    total_epochs  =   50
    batch_size    =  128

    C = 100. # optimum for benchmark model according to sklearn docs: https://scikit-learn.org/stable/auto_examples/neural_networks/plot_rbm_logistic_classification.html#sphx-glr-auto-examples-neural-networks-plot-rbm-logistic-classification-py)

    # --------------------------------------------------------------------------------
    # construct models

    # RBM
    rbm = BernoulliRBM(n_components=total_units, learning_rate=learning_rate, batch_size=batch_size, n_iter=total_epochs, verbose=1)

    # "output layer"
    logistic = LogisticRegression(C=C, solver='lbfgs', multi_class='multinomial', max_iter=200, verbose=1)

    models = []
    models.append(Pipeline(steps=[('logistic', clone(logistic))]))                                              # base model / benchmark
    models.append(Pipeline(steps=[('rbm1', clone(rbm)), ('logistic', clone(logistic))]))                        # single RBM
    models.append(Pipeline(steps=[('rbm1', clone(rbm)), ('rbm2', clone(rbm)), ('logistic', clone(logistic))]))  # RBM stack / DBN

    # --------------------------------------------------------------------------------
    # train and evaluate models

    for model in models:
        # train
        model.fit(X_train, Y_train)

        # evaluate using validation set
        print("Model performance:\n%s\n" % (
            classification_report(Y_val, model.predict(X_val))))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\User\Downloads\DSML MsC\2nd semester\Στοχαστικές Διεργασίες και Βελτιστοποίηση\StochasticsLabPublic\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min finished


Model performance:
              precision    recall  f1-score   support

         0.0       0.95      0.96      0.96       995
         1.0       0.96      0.98      0.97      1121
         2.0       0.90      0.90      0.90      1015
         3.0       0.90      0.88      0.89      1033
         4.0       0.93      0.92      0.92       976
         5.0       0.90      0.88      0.89       884
         6.0       0.94      0.94      0.94       999
         7.0       0.92      0.93      0.92      1034
         8.0       0.88      0.87      0.87       923
         9.0       0.89      0.90      0.90      1020

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000


[BernoulliRBM] Iteration 1, pseudo-likelihood = -139.70, time = 27.41s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -117.34, time = 36.58s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -105.13, time = 32.35s
[Bernoul

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\User\Downloads\DSML MsC\2nd semester\Στοχαστικές Διεργασίες και Βελτιστοποίηση\StochasticsLabPublic\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min finished


Model performance:
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99       995
         1.0       0.99      0.99      0.99      1121
         2.0       0.97      0.98      0.97      1015
         3.0       0.98      0.96      0.97      1033
         4.0       0.98      0.97      0.97       976
         5.0       0.97      0.97      0.97       884
         6.0       0.98      0.98      0.98       999
         7.0       0.98      0.98      0.98      1034
         8.0       0.97      0.96      0.96       923
         9.0       0.95      0.97      0.96      1020

    accuracy                           0.98     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.98      0.98      0.98     10000


[BernoulliRBM] Iteration 1, pseudo-likelihood = -141.65, time = 18.89s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -118.69, time = 30.54s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -109.50, time = 27.62s
[Bernoul

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\User\Downloads\DSML MsC\2nd semester\Στοχαστικές Διεργασίες και Βελτιστοποίηση\StochasticsLabPublic\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min finished


Model performance:
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       995
         1.0       1.00      0.99      0.99      1121
         2.0       0.97      0.98      0.98      1015
         3.0       0.97      0.96      0.97      1033
         4.0       0.98      0.97      0.97       976
         5.0       0.96      0.97      0.97       884
         6.0       0.99      0.98      0.98       999
         7.0       0.98      0.98      0.98      1034
         8.0       0.96      0.96      0.96       923
         9.0       0.95      0.97      0.96      1020

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000





Ένα <b>Restricted Boltzmann Machine (RBM)</b> είναι ένα generative στοχαστικό νευρωνικό δίκτυο το οποίο μαθαίνει την κατανομή των δεδομένων που του δίνονται. Χρησιμοποιούνται εν γένει για dimensionality reduction, feature learning, classification και regression μεταξύ άλλων.

Ονομάζονται Restricted καθώς αποτελούν παραλλαγή των Boltzmann machines, με τη διαφορά ότι οι <b>δύο τύποι νευρώνων, ορατοί (visible) και κρυφοί (hidden)</b> δεν συνδέονται μεταξύ τους για κάθε ομάδα νευρώνων. Δηλαδή, δεν υπάρχουν συνδέσεις (ακμές μεταξύ κόμβων αν φανταστούμε τους νευρώνες ως κόμβους γράφου) για τους visible κόμβους αναμεταξύ τους όπως και για τους hidden κόμβους αναμεταξύ τους παρά μόνον συνδέσεις μεταξύ visible και hidden κόμβων/νευρώνων. Ένα RBM με 3 νευρώνες visible και 4 νευρώνες hidden φαίνεται παρακάτω στη φωτογραφία.
<br>
<img src="files/Restricted_Boltzmann_machine.png">
<br>

Ο περιορισμός αυτός που είναι και η ειδοποιός διαφορά σε σχέση με τα γενικά Boltzmann machines, καθιστά την **εκπαίδευση των RBMs πιο αποδοτική** από αυτή των (γενικών) Boltzmann machines, καθώς αξιοποιούν μεθόδους βασισμένες σε gradient.

Αν υποθέσουμε ένα σύνολο **n** το πλήθος **visible νευρώνων** $V = \{ v_1, v_2, v_3, ..., v_n \}$ και ένα σύνολο **m** το πλήθος **hidden νευρώνων**  $H = \{ h_1, h_2, h_3, ..., h_m \}$, και τα βάρη $w_{ij}$ που ενώνουν τον $i$ visible με τον $j$ hidden νευρώνα. Τότε μπορούμε σε πίνακα να αναπαραστήσουμε τα βάρη των συνδέσεων μεταξύ των visible με των hidden νευρώνων, όπου στις γραμμές έχουμε τους visible και στις στήλες τους hidden νευρώνες.

$W = \begin{bmatrix} w_{11} & w_{12} & w_{13} & ... & w_{1m} \\
w_{21} & w_{22} & w_{23} & ... & w_{2m}  \\
w_{31} & w_{32} & w_{33} & ... & w_{3m} \\
... & ... & ... & ... & ...  \\
w_{n1} & w_{n2} & w_{n3} & ... & w_{nm}  \end{bmatrix}$ = $\begin{bmatrix} 0 & w_{12} & w_{13} & ... & w_{1m} \\
w_{21} & 0 & w_{23} & ... & w_{2m}  \\
w_{31} & w_{32} & 0 & ... & w_{3m} \\
 ... & ... & ... & ... & ...  \\
w_{n1} & w_{n2} & w_{n3} & ... & w_{nm} \end{bmatrix}$

, όπου $w_{ij} = w_{ji}$ για $i \neq j$, δηλαδή ο πίνακας είναι συμμετρικός και $w_{ii} = 0$. Αυτό μεταφράζεται γραφικά σε ένωση των νευρώνων visible και hidden σα να ήταν κόμβοι ενός γράφου μη κατευθυντικού, και όπου ένας κόμβος δε συνδέεται με τον εαυτό του απευθείας μέσω κάποιας ακμής.

Ακόμη, υπάρχουν bias βάρη για τους n visible και m hidden νευρώνες.
Biases για visible νευρώνες: $a_1, a_2, a_3, ..., a_n$
Biases για hidden νευρώνες: $b_1, b_2, b_3, ..., b_m$

Με βάση τα παραπάνω, η **ενέργεια ενός configuration, δηλαδή ενός ζεύγους boolean διανυσμάτων $(\vec{v},\vec{h})$**, μπορεί να υπολογιστεί. Π.χ.  $(\vec{v}=\begin{bmatrix} 1 \\ 0 \\ 0\end{bmatrix},\vec{h}=\begin{bmatrix} 1 \\ 1 \\  0 \\ 0\end{bmatrix})$ για το παράδειγμα της προηγούμενης φωτογραφίας, όπου έχουμε $n=3$ visible nodes και $m=4$ hidden nodes, θα σήμαινε ότι έχουμε μια κατάσταση/configuration όπου είναι ενεργό το 1ο από τα 3 visible units και το 1ο και 2ο από τα hidden units.

Η ενέργεια ορίζεται ως:
$E(\vec{v},\vec{h})=-\sum_{i=1}^{n}a_iv_i-\sum_{j=1}^{m}b_jh_j-\sum_{i=1}^{n}\sum_{j=1}^{m}v_iw_{ij}h_j$
και για το προηγούμενο παράδειγμα θα ήταν:
$E(\vec{v}=\begin{bmatrix} 1 \\ 0 \\ 0\end{bmatrix},\vec{h}=\begin{bmatrix} 1 \\ 1 \\  0 \\ 0\end{bmatrix})=-(α_1v_1+a_2v_2+a_3v_3)-(b_1h_1+b_2h_2+b_3h_3+b_4h_4)-\sum_{i=1}^{n}\sum_{j=1}^{m}v_iw_{ij}h_j = -a_1-b_1-b_2-(w_{11}+w_{12})$

Ομοίως με τα γενικευμένα Boltzmann machines έπεται η **από κοινού κατανομή πιθανότητας** για τα visible και hidden διανύσματα:
$P(\vec{v},\vec{h})=\frac{1}{Z}e^{-E(\vec{v},\vec{h})}$
με το Z να ισούται με $\sum_{\vec{v},\vec{h}}e^{-E(\vec{v},\vec{h})}$ δηλαδή με το άθροισμα των εκθετικών των ενεργειών για όλα τα πιθανά configurations. Αυτό πρακτικά αποτελεί τη σταθερά κανονικοποίησης για την πιθανότητα.

Η περιθώρια κατανομή πιθανότητας για ένα visible διάνυσμα θα είναι επομένως το άθροισμα για όλα τα hidden layer configurations της προηγούμενης από κοινού κατανομής πιθανότητας. Δηλαδή:
$P(\vec{v})= \sum_{\vec{h}}P(\vec{v},\vec{h})=\sum_{\vec{h}}\frac{1}{Z}e^{-E(\vec{v},\vec{h})}=\frac{1}{Z}\sum_{\vec{h}}e^{-E(\vec{v},\vec{h})}$ και ομοίως μιας και είναι bipartite ο γράφος τα hidden units είναι ανεξάρτητα δεδομένου των visible unit activations και προκύπτει για την περιθώρια κατανομή πιθανότητας ενός hidden διανύσματος:
$P(\vec{h})= \sum_{\vec{v}}P(\vec{v},\vec{h})=\sum_{\vec{v}}\frac{1}{Z}e^{-E(\vec{v},\vec{h})}=\frac{1}{Z}\sum_{\vec{v}}e^{-E(\vec{v},\vec{h})}$. Με τον ίδιο τρόπο, τα visible units είναι ανεξάρτητα δεδομένου των hidden unit activations και άρα έχουμε:
$P(\vec{v}|\vec{h})=\prod_{i=1}^{n}P(v_i|\vec{h})$ και
 $P(\vec{h}|\vec{v})=\prod_{j=1}^{m}P(h_j|\vec{v})$

Η περίπτωση να έχουμε activation ενός μόνο νευρώνα i visible ($\vec{v}=[0, 0, ..., 0, v_i, 0, ..., 0]^T$) δεδομένου ενός configuration $\vec{h}$ για το hidden state έχει πιθανότητα:
$P(v_i=1|\vec{h})=P(\vec{v}=[0,0,...,0,v_i​,0,...,0]^T|\vec{h})=\frac{P(\vec{v}=[0,0,...,0,v_i​,0,...,0, \vec{h})}{P(\vec{h})}=\frac{\frac{1}{Z}e^{-E(\vec{v}=[0,0,...,0,v_i​,0,...,0]^T,\vec{h})}}{\frac{1}{Z}\sum_{\vec{v}}e^{-E(\vec{v},\vec{h})}}=\frac{e^{-E(\vec{v}=[0,0,...,0,v_i​,0,...,0]^T,\vec{h})}}{\sum_{\vec{v}}e^{-E(\vec{v},\vec{h})}}$.

Στην περίπτωση αυτή που το $\vec{v}$ έχει μόνο μία μη μηδενική τιμή στη θέση $i$, προκύπτει η ενέργεια:
$E(\vec{v}=[0,0,...,0,v_i​,0,...,0]^T,\vec{h})=-a_i-\sum_{j=1}^{m}w_{ij}h_j$ και άρα:
$P(v_i=1|\vec{h})=σ(a_i + \sum_{j=1}^{m}w_{ij}h_j)$, όπου $σ$ είναι η σιγμοειδής συνάρτηση $σ(x)=\frac{1}{1+e^{-x}}$.

Ο αλγόριθμος εκπαίδευσης για τα RBM αφορά τη μεγιστοποίηση του γινομένου πιθανοφάνειας ενός training set μεγέθους k, $V = \{\vec{v_1}, \vec{v_2}, ..., \vec{v_k}\}$:
$arg max_W \prod_{\vec{u} \in V}P(\vec{v})$
Δηλαδή δίνεται ένα δείγμα εκπαίδευσης με k δειγματικά στοιχεία (διανύσματα). Με βάση τα όσα αναφέραμε προηγουμένως καθένα από αυτά τα k διανύσματα εκπαίδευσης $\vec{v_1}, \vec{v_2}, ..., \vec{v_k}$ έχει διάσταση n (δηλαδή έχουμε n χαρακτηριστικά/features στο dataset), και το RBM κωδικοποιεί αυτές τις n διαστάσεις/χαρακτηριστικά σε m (μέσω των hidden states), που είναι και μια εφαρμογή των RBMs για dimensionality reduction όταν m (hidden states) < n (visible states).

Ο πιο συνήθης αλγόριθμος εκπαίδευσης για τα RBMs, δηλαδή για την εύρεση βέλτιστης τιμής για τον πίνακα βαρών **W**, είναι ο contrastive divergence algorithm που πραγματοποιεί Gibbs sampling και χρησιμοποιείται εντός μιας gradient descent διαδικασίας για την ανανέωση των βαρών. Ο contrastive divergence (CD) algorithm αποτελεί μια προσέγγιση της Maximum Likelihood Estimation (MLE) μεθόδου που θα εφαρμοζόταν ιδανικά για την εκμάθηση των βαρών. Τελικά τα βάρη ανανεώνονται σε ένα RBM ως εξής:

$w_{ij}(t_1)=w_{ij}(t)+η \frac{\partial log(P(\vec{v}))}{\partial w_{ij}}$


<li>Ποια είναι η λογική των <i>DBN</i> και σε τι προβλήματα τα αξιοποιούμε;</li>

Τα Deep Belief Networks (DBNs) είναι generative μοντέλα που αποτελούνται από συνδυασμό πολλαπλών Restricted Boltzmann Machines (RBMs), όπου για κάθε RBM layer που χρησιμοποιείται, το hidden layer του προηγούμενου RBM είναι το visible layer του επόμενου.

Τα DBN εκαπιδεύεται one layer at a time, δηλαδή τα RBMs εκπαιδεύονται το ένα μετά το άλλο ξεκινώντας από το 1o visible layer (layer εισόδου).

Τα διάφορα layers λειτουργούν σαν feature detectors, δηλαδή εξάγουν χαρακτηριστικά από τα δεδομένα μας. Επίσης, ένα DBN μπορεί να εκαπιδευτεί επιβλεπόμενα για classification. Τα DBNs έχουν γενικώς χρησιμοποιηθεί για generation και classification εικόνων, video sequences και motion-capture data.

<li>Να αναφέρετε τις βασικότερες εφαρμογές των <i>RBM</i> και <i>DBN</i>.</li>

Όπως αναφέρθηκε προηγουμένως, τα RBMs είναι δίκτυα που μαθαίνουν την κατανομή των δεδομένων που τους δίνονται. Χρησιμοποιούνται εν γένει για dimensionality reduction, feature learning, classification και regression μεταξύ άλλων και έχουν εφαρμογή σε εικόνες λόγου χάρη.

Ομοίως, τα DBNs που είναι συνδυασμός layers από RBMs, χρησιμοποιούνται για να παράγουν και να κάνουν classify εικόνες, video sequences και motion-capture data.

<li>Εκτός από <i>RBM</i>, τι άλλα μοντέλα μπορούν να χρησιμοποιηθούν για να δημιουργήσουν <i>DBN</i>.</li>

Μια άλλη επιλογή αντί για layers από RBMs για τη δημιουργία DBNs είναι οι autoencoders.

<li>Συγκρίνετε τα αποτελέσματα της ταξινόμησης με τον αλγόριθμo <i>Logistic Regression</i> χωρίς τη χρήση <i>RBM</i> σε σχέση με τα αποτελέσματα της ταξινόμησης που έχει χρησιμοποιηθεί η <i>RBM</i> καθώς και με αυτή όπου χρησιμοποιούνται <i>RBM</i> και <i>DBN</i> για την εξαγωγή των χαρακτηριστικών. Τι παρατηρείτε ως προς την ακρίβεια των αποτελεσμάτων;</li>

Με βάση τα αποτελέσματα της ταξινόμησης, βλέπουμε πως η ταξινόμηση με RBM και DBN βελτιώνει σημαντικά την ταξινόμηση. Ενώ ίσως θα περιμέναμε μια κάποια βελτίωση χρησιμοποιώντας DBN αντί RBM, δεν είδαμε κάτι τέτοιο τελικά. Βέβαια, οι μετρικές ήταν ήδη πολύ υψηλά για το RBM. Βλέπουμε λοιπόν πως η αναπαράσταση των αρχικών χαρακτηριστικών με νέα μέσω ενός (απλό RBM) ή δύο (DBN) RBM layers επιφέρει σημαντική βελτίωση στις μετρικές (πχ accuracy).

References:
- [Medium - What Are RBMs, Deep Belief Networks and Why Are They Important to Deep Learning?](https://medium.com/swlh/what-are-rbms-deep-belief-networks-and-why-are-they-important-to-deep-learning-491c7de8937a)
- [YouTube - Restricted Boltzmann Machines (RBM) - A friendly introduction](https://www.youtube.com/watch?v=Fkw0_aAtwIw)
- [Wikipedia - Restricted Boltzmann machine](https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine)
- [Wikipedia - Boltzmann machine](https://en.wikipedia.org/wiki/Boltzmann_machine)
- [Wikipedia - Deep belief network](https://en.wikipedia.org/wiki/Deep_belief_network)